In [6]:
from utils.train import Create, Build
from utils.process import Format, Preprocess, Categorize, Assemble
from utils.generate import Generate
from utils.test import Test
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
generate = Generate()
categorize = Categorize()
assemble = Assemble()
test = Test()

# Regression

In [15]:
models = [[{'model': 'LR1', 'parameters': {}}, {'feature_selection': ['SFM1', {}], 'dimensionality_reduction': ['PCA1', {}]}],
          [{'model': 'RFR1', 'parameters': {'n_estimators': 20, 'criterion':'squared_error', 'verbose':1}}, 
           {'feature_selection': ['RFE1', {'n_features_to_select': 3, 'step':1}], 'dimensionality_reduction': ['PCA1', {}]}]
         ]   

In [12]:
df = pd.read_csv('./data/Clean_Dataset.csv')

df['path'] = df['source_city'] + ' ' + df['destination_city']
df['time'] = df['departure_time'] + ' ' + df['arrival_time']
df.replace({'Economy':0,'Business':1}, inplace=True)
df = df.drop(['Unnamed: 0','flight','source_city','destination_city','departure_time','arrival_time'], axis=1)

days_left_ranges = generate.generate_numerical_ranges(df.days_left, [2,5,10,18])
days_left_dict = categorize.categorize_numerical_variable(df.days_left, days_left_ranges)
df['days_left_category'] = df['days_left'].map(days_left_dict)

start = df.groupby('path').mean()['price'].min()
finish = df.groupby('path').mean()['price'].max()
path_ranges = generate.generate_categorical_ranges(3, start, finish)


path_group = df.groupby('path').mean()['price']
path_dict = categorize.categorize_categorical_variable(path_group, path_ranges)
df['path_category'] = df['path'].map(path_dict)

duration_ranges = generate.generate_numerical_ranges(df.duration, [5,18,28])
duration_dict = categorize.categorize_numerical_variable(df.duration, duration_ranges)
df['duration_category']=df['duration'].map(duration_dict)

start = df.groupby('time').mean()['price'].min()
finish = df.groupby('time').mean()['price'].max()
time_ranges = generate.generate_categorical_ranges(3, start, finish)


time_group = df.groupby('time').mean()['price']
time_dict = categorize.categorize_categorical_variable(time_group, time_ranges)
df['time_category'] = df['time'].map(time_dict)

df.drop(['days_left','path','duration','time'], axis=1, inplace=True)

In [16]:
all_regression_test_dict = {}
all_regression_test_dict['data'] = {'A1':df}

In [ ]:
build = Build(all_regression_test_dict, feature_selection=True, dimensionality_reduction=True)
build.build_regression_models(models, 'price')

Selecting features with SFM1 for A1
Reducing dimensions with PCA1 for A1
Training regression model LR1 for A1
Training done!

Selecting features with RFE1 for A1


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   10.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   10.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    9.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    9.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [11]:
all_test_tables_dict = assemble.assemble_test_tables(all_regression_test_dict)
all_regression_test_dict['test_tables'] = all_test_tables_dict
assemble.assemble_error_values(all_regression_test_dict)

,MEPE,MPE,MEAE,MAE,MSE,RMSE,NRMSE,STD
A1LR1SFM1PCA1,20.776,35.326,2158.929,4437.361,5.671149e+07,7530.703,0.066291,22612.54
A1RFR1RFE1PCA1,25.540,37.092,2499.899,4037.789,3.730425e+07,6107.721,0.053765,22612.54


In [8]:
#After Update
all_test_tables_dict = assemble.assemble_test_tables(all_regression_test_dict)
all_regression_test_dict['test_tables'] = all_test_tables_dict
assemble.assemble_error_values(all_regression_test_dict)

,MEPE,MPE,MEAE,MAE,MSE,RMSE,NRMSE,STD
A1LR1SFM1PCA1,20.776,35.326,2158.929,4437.361,5.671149e+07,7530.703,0.066291,22612.54
A1RFR1RFE1PCA1,25.516,37.107,2503.399,4038.549,3.730475e+07,6107.762,0.053766,22612.54


In [36]:
all_regression_test_dict['predictions']

{'A1LR1SFM1PCA1': array([ 8020.60831351, 51247.63261608,  8020.60831351, ...,
         5331.07079847,  8020.60831351,  5331.07079847]),
 'A1RFR1RFE1PCA1': array([ 8135.77092956, 57830.05053253,  3994.40825619, ...,
         6360.39852885,  6360.39852885,  3994.40825619])}

# Classification

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.datasets import load_digits

In [3]:
iris = load_iris()
digits = load_digits()

In [4]:
dfi = pd.DataFrame(data= np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target'])
dfd = pd.DataFrame(data= np.c_[digits['data'], digits['target']], columns= digits['feature_names'] + ['target'])

In [5]:
models = [[{'model': 'LR1', 'parameters': {}}, {'feature_selection': ['SFM1', {}], 'dimensionality_reduction': ['PCA1', {}]}],
          [{'model': 'RFC1', 'parameters': {'n_estimators': 20, 'criterion':'gini'}}, 
           {'feature_selection': ['RFE1', {'n_features_to_select': 3, 'step':1}], 'dimensionality_reduction': ['PCA1', {}]}]
         ]   

In [6]:
all_classification_test_dict = {}
all_classification_test_dict['data'] = {'A1':dfi, 'A2':dfd}

In [7]:
build = Build(all_classification_test_dict)
build.build_classification_models(models, 'target')
assemble = Assemble()

Training classification model LR1 for A1
Training done!

Training classification model RFC1 for A1
Training done!

Training classification model LR1 for A2


C:\Users\melis\anaconda3\envs\ml\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training done!

Training classification model RFC1 for A2
Training done!



In [8]:
assemble.assemble_classification_report(all_classification_test_dict)

,CLASS,PRECISION,RECALL,F1-SCORE,SUPPORT,ACCURACY
ALGORITHM,,,,,,
A1LR1,0.0,1.000000,1.000000,1.000000,11.0,0.900000
A1LR1,1.0,0.916667,0.846154,0.880000,13.0,0.900000
A1LR1,2.0,0.714286,0.833333,0.769231,6.0,0.900000
A1RFC1,0.0,1.000000,1.000000,1.000000,11.0,0.966667
A1RFC1,1.0,0.928571,1.000000,0.962963,13.0,0.966667
A1RFC1,2.0,1.000000,0.833333,0.909091,6.0,0.966667
A2LR1,0.0,1.000000,1.000000,1.000000,27.0,0.961111
A2LR1,1.0,0.911765,0.885714,0.898551,35.0,0.961111
A2LR1,2.0,0.972973,1.000000,0.986301,36.0,0.961111


In [1]:
pwd

'C:\\Users\\melis\\Desktop\\DataScience\\SupervisedLearning'